In [23]:
import numpy as np
import pandas as pd

----------------- Simulating Click Data For A/B Test -----------------

In [24]:
N_exp = 10000
N_con = 10000

In [25]:
# Generating Click Data
clicks_exp = pd.Series(np.random.binomial(1, 0.5, N_exp))
clicks_con = pd.Series(np.random.binomial(1, 0.2, N_con))

In [26]:
# Generate Group Identifier
exp_id = pd.Series(np.repeat("exp", N_exp))
con_id = pd.Series(np.repeat("con", N_con))

In [27]:
df_exp = pd.concat([clicks_con, con_id], axis=1)
df_con = pd.concat([clicks_exp, exp_id], axis=1)

In [28]:
df_exp.columns = ["click", "group"]
df_con.columns = ["click", "group"]
print(df_exp)
print(df_con)

      click group
0         0   con
1         0   con
2         1   con
3         0   con
4         0   con
...     ...   ...
9995      0   con
9996      0   con
9997      0   con
9998      0   con
9999      0   con

[10000 rows x 2 columns]
      click group
0         0   exp
1         1   exp
2         1   exp
3         0   exp
4         0   exp
...     ...   ...
9995      1   exp
9996      0   exp
9997      1   exp
9998      1   exp
9999      0   exp

[10000 rows x 2 columns]


In [29]:
df_ab_test = pd.concat([df_exp, df_con]).reset_index(drop=True)
print(df_ab_test)

       click group
0          0   con
1          0   con
2          1   con
3          0   con
4          0   con
...      ...   ...
19995      1   exp
19996      0   exp
19997      1   exp
19998      1   exp
19999      0   exp

[20000 rows x 2 columns]


In [30]:
# total clicks for control and experimental groups

print(df_ab_test.groupby("group")['click'].sum())

x_con = df_ab_test.groupby("group")['click'].sum().loc["con"]
x_exp = df_ab_test.groupby("group")['click'].sum().loc["exp"]

group
con    1955
exp    5033
Name: click, dtype: int32


In [31]:
# Calculate the click probabilities of control and experimental groups

p_con_hat = x_con / N_con
p_exp_hat = x_exp / N_exp 

print("Click probabilities of control group: ", p_con_hat)
print("Click probabilities of experimental group: ", p_exp_hat)

Click probabilities of control group:  0.1955
Click probabilities of experimental group:  0.5033


In [35]:
# Pooled Success Probability

p_pooled_hat = (x_con + x_exp) / (N_con + N_exp)
print("Pooled Success Probability (p^): ", p_pooled_hat)

Pooled Success Probability (p^):  0.3494


In [34]:
# Pooled Variance

var_pooled_hat = p_pooled_hat * (1 - p_pooled_hat) * (1 / N_con + 1 / N_exp)
print("Pooled Variance: ", var_pooled_hat)

Pooled Variance:  4.546392800000001e-05


In [36]:
# Standard Error

se_pooled_hat = np.sqrt(var_pooled_hat)
print("Standard Error: ", se_pooled_hat)

Standard Error:  0.00674269441692266


In [37]:
# calculate test statistics

Test_stat = (p_con_hat - p_exp_hat) / se_pooled_hat
print("Test Statistic for 2 sample z-test: ", Test_stat)

Test Statistic for 2 sample z-test:  -45.64940674569065


In [40]:
# Calculate z critical

from scipy.stats import norm

alpha = 0.05
z_critical = norm.ppf(1 - alpha/2)
print("Z Critical Value from standard Normal Distribution: ", z_critical)

Z Critical Value from standard Normal Distribution:  1.959963984540054


In [42]:
# Calculate P value

p_value = 2 * (1 - norm.cdf(abs(Test_stat)))
print("P Value: ", p_value)

# p_value = 2 * norm.sf(abs(Test_stat))

P Value:  0.0


In [45]:
# Check if the test is statistically significant

if p_value < alpha:
    print("We reject the null hypothesis")
    print("There is a statistically significance difference between control and experimental versions")
else:
    print("We fail to reject the null hypothesis")
    print("There is no statistically significance difference between control and experimental versions")


We reject the null hypothesis
There is a statistically significance difference between control and experimental versions


In [50]:
# Calculate confidence interval

ci = np.array([round((p_con_hat - p_exp_hat) - z_critical * se_pooled_hat, 3) , round((p_con_hat - p_exp_hat) + z_critical * se_pooled_hat,3)])
print("Confidence Interval: ", ci)


Confidence Interval:  [-0.321 -0.295]


In [68]:
# Check if the test is Practically significant

def check_if_practical_significant(delta):
    if delta < ci[0]:
        print("The test is practically significant")
    else:
        print("It's possible that the change is not practically significant")

In [69]:
check_if_practical_significant(-0.4)

The test is practically significant


In [70]:
check_if_practical_significant(0.32)

It's possible that the change is not practically significant
